In [6]:
import numpy as np
from math import factorial
from numpy.linalg import norm, matrix_power
from scipy.linalg import expm
import time
from scipy.interpolate import pade
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interactive
import import_ipynb
import Error_Analysis_curves as error_anal
from math import pow as power

In [7]:
def taylor_exp(A, tol = 1e-8, p = 2): 
    
    start = time.perf_counter()
    n = len(A)
    print(f'Dim of A n: {n}\n')
    print(f'The Matrix A:\n{A}\n')
    
    norm_A = norm(A,ord = p) #2 norm

    #s = 0
    s = int(np.ceil(np.log2(norm_A)+1))  # by solving ||A||/2^s < 1
    print(f'The value of squaring size s: {s}\n')

    B = A/2**s

    k = 0
    M0 = np.eye(n)  #identity matrix 

    while norm(matrix_power(B,k),ord = p)/factorial(k) > tol:
        #print(norm(matrix_power(B,k),ord = p)/factorial(k))
        #print(*M0)
        #print(matrix_power(B,k)/factorial(k))
        k+=1
        M0 = np.add(M0,matrix_power(B,k)/factorial(k))
        
        

    end = time.perf_counter()    
    print('DONE')
    print(f'We truncated our Taylor Series to m = {k-1} terms where m(tol)\n') #by the code we add calculate dimension of T_m not degree

    M = matrix_power(M0,2**s)
    exact_M = expm(A)

    print(f'Actual e^A using expm function:\n{exact_M}\n')
    print(f'Estimation :\n{M}\n')
    
    error = norm(M-exact_M,ord=p)/(norm(exact_M,ord = p))
    
    print(f'Relative Forward Error: {error}')
    print(f'Wrong by: {round(np.log10(error))} digits') #if we have negative val then its accurate to that many digits
    print(f'Time required for computation: {end-start} seconds')
    print('===============================================================')
    
    return error,M,s,k  #here my m = k

if __name__ == '__main__':
    n = 2
    tol = 0.5*np.spacing(1)
    A = np.array([[-49,24],[-64,31]])
    norm_A = norm(A,ord = 2) #2 norm
    print(tol*norm_A*np.exp(tol*norm_A))
    #A = np.random.rand(n,n) + 1j*np.random.rand(n,n)
    #A = np.random.randint(0,9,(n,n))
    #print(help(norm))
    taylor_exp(A,tol = tol, p=2)
    

9.9511935020855e-15
Dim of A n: 2

The Matrix A:
[[-49  24]
 [-64  31]]

The value of squaring size s: 8

DONE
We truncated our Taylor Series to m = 9 terms where m(tol)

Actual e^A using expm function:
[[-0.73575876  0.5518191 ]
 [-1.4715176   1.10363824]]

Estimation :
[[-0.73575876  0.5518191 ]
 [-1.4715176   1.10363824]]

Relative Forward Error: 1.651983806142441e-13
Wrong by: -13.0 digits
Time required for computation: 0.001609455999997067 seconds


In [8]:
def taylor_exp(A, m = 10, s = 2,p = 2): 
    
    start = time.perf_counter()
    n = len(A)
    print(f'Dim of A n: {n}\n')
    print(f'The Matrix A:\n{A}\n')
    eigval,X = np.linalg.eig(A)
    cond = np.linalg.cond(X,p)
    print(f'Eigenvalues of A: {eigval}')
    print(f'Real part of spectral radius of A: {np.real(eigval[0])}')
    print(f'Condition Number of eigenvector: {cond}\n')
    norm_A = norm(A,ord = p) #2 norm

    #s = 0
#     s = int(np.ceil(np.log2(norm_A)+1))  # by solving ||A||/2^s < 1
#     print(f'The value of squaring size s: {s}\n')

    #B = A/2**s

    k = 0
    M0 = np.eye(n)  #identity matrix 

    for k in range(1,m+1):
        M0 += matrix_power(A,k)/(power(2,s*k)*factorial(k))
        
    end = time.perf_counter()    
    print('DONE')
    print(f'We truncated our Taylor Series to m = {k-1} terms where m(tol)\n') #by the code we add calculate dimension of T_m not degree

    M = matrix_power(M0,2**s)
    exact_M = expm(A)

    print(f'Actual e^A using expm function:\n{exact_M}\n')
    print(f'Estimation :\n{M}\n')
    
    rel_error = norm(M-exact_M,ord= p)/(norm(exact_M,ord = p))
    abs_error = norm(M-exact_M,ord= p)
    
    print(f'Absolute Error: {abs_error}')
    print(f'Relative Error: {rel_error}')
    print(f'Wrong by: {np.log10(rel_error)} digits') #if we have negative val then its accurate to that many digits
    print(f'Time required for computation: {end-start} seconds')
    print('===============================================================')
    
    return abs_error,rel_error,eigval, cond  #here my m = k

if __name__ == '__main__':
    n = 2  #size of the matrix
    m = 10 #truncation parameter
    s = 8 #scaling parameter
    p = 2 #norm
    A = np.array([[49,24],[-64,31]])
    
    absolute, rel, eigval, cond = taylor_exp(A,m,s, p)

Dim of A n: 2

The Matrix A:
[[ 49  24]
 [-64  31]]

Eigenvalues of A: [40.+38.14446225j 40.-38.14446225j]
Real part of spectral radius of A: 40.0
Condition Number of eigenvector: 1.7284740252748652

DONE
We truncated our Taylor Series to m = 9 terms where m(tol)

Actual e^A using expm function:
[[ 100.73818873  133.75156516]
 [-372.63087481  -12.55189738]]

Estimation :
[[ 2.36350012e+17  6.38026634e+16]
 [-1.70140440e+17  1.88498014e+17]]

Absolute Error: 3.008043512244424e+17
Relative Error: 772892374729437.6
Wrong by: 14.88811902261779 digits
Time required for computation: 0.0013977249999896912 seconds


In [98]:
R = abs(np.real(eigval[0]))
Z = error_anal.random_points(R,200)
Exact = np.exp(Z)
type = 'rel'

T_m_s = (error_anal.approx_taylor_exp(Z/(2**s),m))**(power(2,s))   #composite poly 
error = error_anal.rel_error(T_m_s, Exact,type)  #calculating the log10 rel error
max_idx = np.argmax(error)
max_error = max(error)  #choosing the maximum one
idx = np.argmax(error)
print(f'Radius R = {R}')
print(f'max at z = {Z[max_idx]}')
print(f'Max error: {cond*10**(max_error)}')
print(f'Max {type} error = {np.log10(cond)+max_error}')

Radius R = 495.694603317893
max at z = (-492.0107433786022-60.320545078095286j)
Max error: 7.722811927008669e-13
Max rel error = -12.112224541369953
